In [26]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
URL="https://www.dlsu.edu.ph"

In [3]:
page = requests.get(URL)

# Extract all sub-urls from dlsu.edu.ph

In [14]:
import requests
import html5lib
from bs4 import BeautifulSoup
import re
import time

soup = BeautifulSoup(page.content, 'lxml')
data = []
links = []

url_limit = 50

start = time.time()

def remove_duplicates(l): # remove duplicates and unURL string
    for item in l:
        match = re.search("(?P<url>https?://[^\s]+)", item)
        if match is not None:
            links.append((match.group("url")))


for link in soup.find_all('a', href=True):
    data.append(str(link.get('href')))
    
flag = True
remove_duplicates(data)

while flag:
    try:
        for link in links:
            for j in soup.find_all('a', href=True):
                temp = []
                source_code = requests.get(link)
                soup = BeautifulSoup(source_code.content, 'lxml')
                temp.append(str(j.get('href')))
                remove_duplicates(temp)

                if len(links) > url_limit: # set limitation to number of URLs
                    break
            if len(links) > url_limit:
                break
        if len(links) > url_limit:
            break
    except Exception as e:
        print(e)
        if len(links) > url_limit:
            break
            
for url in links:
    print(url)
    
print ("Elapsed Time: %s" % (time.time() - start))

https://www.dlsu.edu.ph/about-dlsu/
https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/
https://www.dlsu.edu.ph/offices/qao/accreditation/
https://www.dlsu.edu.ph/offices/office-of-the-chancellor/
https://www.dlsu.edu.ph/university-fellows/
https://www.dlsu.edu.ph/emeriti/
https://www.dlsu.edu.ph/brothers/
https://www.dlsu.edu.ph/lasallianmission/
https://www.dlsu.edu.ph/offices/
https://www.dlsu.edu.ph/inside/board-advisors/
https://www.dlsu.edu.ph/admissions/scholarships/
https://www.dlsu.edu.ph/admissions/graduate/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/
https://www.dlsu.edu.ph/colleges/
https://www.dlsu.edu.ph/colleges/gcoe/overview/
http://www.dlsu.edu.ph/wp-content/uploads/pdf/laguna-campus/dlsu-laguna-campus-viewbook.pdf
https://www.dlsu.edu.ph/admissions/graduate/graduate-viewbooks/
https://www.dlsu.edu.ph/wp-content/uploads/pd

# Get Name & Email

In [15]:
def deCFEmail(fp):
    try:
        r = int(fp[:2],16)
        email = ''.join([chr(int(fp[i:i+2], 16) ^ r) for i in range(2, len(fp), 2)])
        return email
    except (ValueError):
        pass

In [16]:
emailList = []
nameList = []
urlList = []

def emailNameExtractor(urlString, url):
    getH=requests.get(urlString)
    h=getH.content
    soup=BeautifulSoup(h,'html.parser')
    mailtos = soup.select('a[href]')
    for i in mailtos:
        href=i['href']
        if "email-protect" in href:
            emailList.append(deCFEmail(href.split("#")[1]))
            nameList.append(i.text)
            urlList.append(url)

In [17]:
a = 'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'
links.append(a)

In [18]:
emailList = []
nameList = []
urlFrom = []

length = 0
prev_length =  0



for x in range(len(links)):
    prev_length = length
    length = len(emailList)
    
    if prev_length != length: 
        print("Email Length: ",len(emailList), "Name Length: ", len(nameList), "index: ", x)
        
    
    print(links[x])
    
    if links[x].lower().endswith(".pdf"):
        pass
    else:
        try:
            emailNameExtractor(links[x], links[x])
        except:
            pass
   
            
emailList
nameList

https://www.dlsu.edu.ph/about-dlsu/
https://www.dlsu.edu.ph/about-dlsu/ourcorevalues/
https://www.dlsu.edu.ph/offices/qao/accreditation/
Email Length:  1 Name Length:  1 index:  3
https://www.dlsu.edu.ph/offices/office-of-the-chancellor/
Email Length:  2 Name Length:  2 index:  4
https://www.dlsu.edu.ph/university-fellows/
https://www.dlsu.edu.ph/emeriti/
https://www.dlsu.edu.ph/brothers/
https://www.dlsu.edu.ph/lasallianmission/
https://www.dlsu.edu.ph/offices/
https://www.dlsu.edu.ph/inside/board-advisors/
https://www.dlsu.edu.ph/admissions/scholarships/
https://www.dlsu.edu.ph/admissions/graduate/
Email Length:  4 Name Length:  4 index:  12
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integrated-school/
https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/
Email Length:  10 Name Length:  10 index:  16
https://www.dlsu.edu.ph/colleges/
https://www.dlsu.edu.ph/colleges/gcoe/overview/
http://www.dlsu.edu.ph/wp

https://www.dlsu.edu.ph/contact-us-new-normal/
Email Length:  54 Name Length:  54 index:  122
https://www.dlsu.edu.ph/dlsu-care-desk/
https://www.dlsu.edu.ph/dlsu-care-desk/
https://www.dlsu.edu.ph/covid-19-resource/
https://www.dlsu.edu.ph/covid-19-resource/
https://www.dlsu.edu.ph/connectivity-support/
https://www.dlsu.edu.ph/connectivity-support/
https://www.dlsu.edu.ph/dlsu-return-to-campus-guidelines/
https://www.dlsu.edu.ph/dlsu-return-to-campus-guidelines/
https://www.dlsu.edu.ph/lasallians-r-e-a-c-h/
https://www.dlsu.edu.ph/lasallians-r-e-a-c-h/
https://www.dlsu.edu.ph/list-of-important-help-desk-announcements/
https://www.dlsu.edu.ph/list-of-important-help-desk-announcements/
https://www.dlsu.edu.ph/give/
https://www.dlsu.edu.ph/animo-christmas-2022/
https://www.dlsu.edu.ph/animo-christmas-2022/
https://www.dlsu.edu.ph/online-request-for-documents/
https://www.dlsu.edu.ph/online-request-for-documents/
https://www.dlsu.edu.ph/ban-of-single-use-plastics-sups-on-campus/
https://w

['[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',
 '[email\xa0protected]',


# Combined Email and Name List

In [19]:
urlList

['https://www.dlsu.edu.ph/offices/qao/accreditation/',
 'https://www.dlsu.edu.ph/offices/office-of-the-chancellor/',
 'https://www.dlsu.edu.ph/admissions/graduate/',
 'https://www.dlsu.edu.ph/admissions/graduate/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/integratedschool-test/senior-high-school/',
 'https://www.dlsu.edu.ph/offices/hub/cav-ched-requests/',
 'https://www.dlsu.edu.ph/research/research-centers/shore/',
 'https://www.dlsu.edu.ph/research/research-centers/shore/',
 'https://www.dlsu.edu.ph/research/research-centers/bnscwc/',
 'https://www.dlsu.edu.ph/research/research-centers/bnscwc/',
 'https://www.dlsu.edu.ph/research/research-centers/cesdr/',

In [21]:
finalList = list(zip(nameList,emailList,urlList))
finalList = set(finalList)
finalList

{('Dr. Briane Paul V. Samson',
  'briane.samson@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Charibeth K. Cheng',
  'charibeth.cheng@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Christine Diane L. Ramos',
  'christine.diane.ramos@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Ethel C. Ong',
  'ethel.ong@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Ma. Rowena R. Caguiat',
  'ma.rowena.caguiat@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Marnel S. Peradilla',
  'marnel.peradilla@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Michelle Renee D. Ching',
  'michelle.ching@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/'),
 ('Dr. Rafael A. Cabredo',
  'rafael.cabredo@dlsu.edu.ph',
  'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/')

In [25]:
a = 'https://www.dlsu.edu.ph/research/research-centers/bnscwc/'
testpage = requests.get(a)
soup = BeautifulSoup(testpage.content, 'lxml')

soup

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en-US">
<head><link href="https://www.dlsu.edu.ph/wp-content/cache/minify/91abb.default.include.18995b.css" media="all" rel="stylesheet"/>
<script defer="" src="https://www.dlsu.edu.ph/wp-content/cache/minify/91abb.default.include.51581c.js"></script>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.dlsu.edu.ph/xmlrpc.php" rel="pingback"/>
<title>Bienvenido N. Santos Creative Writing Center - De La Salle University</title>
<!-- This site is optimized with the Yoast SEO plugin v11.6 - https://yoast.com/wordpress/plugins/seo/ -->
<link href="https://www.dlsu.edu.ph/research/research-centers/bnscwc/" rel="canonical"/>
<meta content="en_US" property="og:locale"/>
<meta content="article" property="og:type"/>
<meta content="Bienvenido N. Santos Creative Writing Center - De La Salle University" pr

# Convert to txt file

In [31]:
df = pd.DataFrame (finalList, columns = ['Name', 'Email','url'])
np.savetxt(r'np.txt', df.values, fmt='%s %s %s')

# Threading 

## wala pang ginagawa to

In [12]:
import threading
import time

start = time.time()

def fetch_url(url):
    print ("'%s\' fetched in %ss" % (url, (time.time() - start)))
    
threads = [threading.Thread(target=fetch_url, args=(url,)) for url in links]

for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

print ("Elapsed Time: %s" % (time.time() - start))

'https://www.dlsu.edu.ph/offices/' fetched in 0.0020012855529785156s
'https://www.dlsu.edu.ph/soll/' fetched in 0.004004478454589844s
'https://www.facebook.com/DLSU.GradStudies' fetched in 0.007005929946899414s'https://www.dlsu.edu.ph/give/' fetched in 0.008007287979125977s

'https://www.dlsu.edu.ph/research/research-centers/cesdr/' fetched in 0.01101827621459961s
'https://www.dlsu.edu.ph/research/research-centers/' fetched in 0.013028621673583984s'https://www.linkedin.com/school/de-la-salle-university/' fetched in 0.015024662017822266s

'https://www.dlsu.edu.ph/shs-research-congress/%20' fetched in 0.017004728317260742s
'https://www.dlsu.edu.ph/colleges/ccs/office-of-the-dean/' fetched in 0.02000141143798828s
'https://twitter.com/DLSUGradStudies' fetched in 0.022000789642333984s'https://www.dlsu.edu.ph/wp-content/uploads/2022/06/the-concierge.pdf' fetched in 0.023999929428100586s

'https://www.dlsu.edu.ph/research/research-centers/bnscwc/' fetched in 0.026002883911132812s
'https://www

# Get emails protected by cloudflare

# Pwede na ata di isama to kasi ayaw gumana

In [13]:
mydivs = soup.findAll("span",{"class": "__cf_email__"})

length = len(mydivs)

for x in range(length):
    a = cfDecodeEmail(mydivs[x]["data-cfemail"])
    print(a)


NameError: name 'cfDecodeEmail' is not defined

In [ ]:
print (mydivs[1]["data-cfemail"])
print(len(mydivs))